In [21]:
class TwitterCredentials():
    def __init__(self):
        self.ACCESS_TOKEN = ''
        self.ACCESS_TOKEN_SECRET = ''
        self.CONSUMER_KEY = ''
        self.CONSUMER_SECRET = ''

In [22]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time
import os

import pandas as pd
import numpy as np

class TwitterClient():
    def __init__(self, twitter_user = None):
        self.auth = TwiterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
    
        self.twitter_user = twitter_user

class TwiterAuthenticator():
        twitter_credentials = TwitterCredentials()
        def authenticate_twitter_app(self):
            auth = OAuthHandler(self.twitter_credentials.CONSUMER_KEY, self.twitter_credentials.CONSUMER_SECRET)
            auth.set_access_token(self.twitter_credentials.ACCESS_TOKEN, self.twitter_credentials.ACCESS_TOKEN_SECRET)
            return auth

class TwitterListener(StreamListener):
    #This is a basic listener that just prints received tweets to stdout 
    def __init__(self, fetched_tweets_filename, time_limit):
        self.fetched_tweets_filename = fetched_tweets_filename
        self.start_time = time.time()
        self.time_limit = time_limit
        if os.path.exists(self.fetched_tweets_filename):
            os.remove(self.fetched_tweets_filename)

    def on_data(self, data):
        if (time.time() - self.start_time) < self.time_limit:
            try:
                #print(data)
                with open(self.fetched_tweets_filename, 'a') as tf:
                    tf.write(data)
                return True
            except Exception as e:
                print('Encountered Exception Tweet:', e)
            return True
        else:
            print('Streaming stopped after: ',self.time_limit, 'sec')
            return False

    def on_error(self, status):
        if status == 420:
            # Return false in case rate limit occurs
            return False
        print(status)

class TwitterStreamer():
    #Class for streaming and processing live tweets
    def __init__(self):
        self.twitter_authenticator = TwiterAuthenticator()

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list, time_limit):
        # This handles Twitter authentication and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename, time_limit)
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)
        #This line filters Twitter Streams to capture data by the keywords
        #stream.filter(track = hash_tag_list)       
        stream.sample()

#Run streaming tweets
streaming_time = float(input("Please enter the time that you want to listen to tweets (time in seconds): ")) 
print('Listening Tweets. Please Wait...')
#use hash_tag_list in case of filtering tweets
hash_tag_list = ['donald trump', 'barack obama', 'hillary clinton']
fetched_tweets_filename = 'tweet.json'

twitter_client = TwitterClient()
#print(twitter_client.get_user_timeline_tweets(1))

twitter_streamer = TwitterStreamer()

twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list, streaming_time)

Please enter the time that you want to listen to tweets (time in seconds): 200
Listening Tweets. Please Wait...
Streaming stopped after:  200.0 sec


In [23]:
#Write tweets on output_tweets.csv file
import json
import csv

class TweetStories():
    def record_tweet_stories(self):
        count = 0
        tweets_list = []
        #use this set as ground truth for distinct tweet id's
        distinct_tweets = set()
        filePath = 'C:/Users/iTTaste/tweet.json'

        fo = open(filePath, "r")
        lines = fo.readlines()
        
        #print('Total tweets: ',len(lines))
        for line in lines:
            try:
                tweet = json.loads(line)
                if 'retweeted_status' in tweet:
                    distinct_tweets.add(tweet['retweeted_status']['id'])
                    tweets_list.append([tweet['retweeted_status']['id'],count])
                    count += 1
                elif 'delete' in tweet:
                    pass
                else:
                    tweets_list.append([tweet['id'],count])
                    distinct_tweets.add(tweet['id'])
                    count += 1
            except ValueError:
                pass

        fo.close()
        print('Unique Tweets:',len(distinct_tweets))
        return tweets_list
    
    def write_tweet_stories_on_csv(self, tweets):
        #write tweet id's to a csv file
        with open('output_tweets.csv','w') as f:
            for tweet in tweets:
                f.write(str(tweet[0]) + ',' + str(tweet[1]))
                f.write('\n')


In [24]:
import random

class MakeHashFunction():
    # return a random hash function from a universal family
    # p = prime number > m
    def __init__(self, p=2**33-355, m=2**32-1):
        a = random.randint(1,p-1)
        b = random.randint(0, p-1)
        self.func = lambda x: ((a * x + b) % p) % m
        self.zero_counter = 0

In [25]:
#Run FM algorithm on tweet stories
from statistics import median, mean
import binascii
import csv
import numpy as np

phi = .77351
class AddHashFunction():
    def __init__(self, inner_group):
        self.hash_functions = []
        for i in range(inner_group):
            self.hash_functions.append(MakeHashFunction())

class FMCounting():
    #group_num is the outer group
    def __init__(self, outer_group, inner_group):
        self.group_hash = []
        for i in range(outer_group):
            self.group_hash.append(AddHashFunction(inner_group))

    def count_trailing_zeros(self,n):
        s = str("{:032b}".format(n))
        return len(s)-len(s.rstrip('0'))

    def hash_items(self, elem):
        for g in self.group_hash:
            for f in g.hash_functions:
                hf = f.func(elem)
                current_zeros_len = self.count_trailing_zeros(hf)
                if current_zeros_len > f.zero_counter:
                    f.zero_counter = current_zeros_len
                
    def count_distinct_elems(self):
        group_median = []
        for g in self.group_hash:
            temp = []
            for f in g.hash_functions:
                temp.append(2 ** f.zero_counter)
            group_median.append(median(temp))   
        return round(mean(group_median))
    
a = 10
b = 10
fm = FMCounting(a,b)

#Manipulate tweet classes
tweets = TweetStories()
tweet_list = tweets.record_tweet_stories()
tweets.write_tweet_stories_on_csv(tweet_list)

tweet_ids = []
with open('output_tweets.csv','r') as f:
    data = csv.reader(f)
    for row in data:
        tweet_ids.append(row[0])

data = tweet_ids

for line in data:
            hash_value = binascii.crc32(line.encode('utf-8')) & 0xffffffff
            fm.hash_items(hash_value)
            
print('Stream Length:',len(data))
print('FM Estimation:',fm.count_distinct_elems())

Unique Tweets: 5241
Stream Length: 5374
FM Estimation: 5632


In [38]:
#Trial Using Reservoir Sampling
#AMS Algorithm k-moment for k = 2 
import random
import string
import binascii
import operator
from collections import Counter

N = 0
K = 2500

tweets = []
data = []

#Read tweet id's from csv file
with open('output_tweets.csv','r') as f:
    contents = csv.reader(f)
    for row in contents:
        tweets.append(row[0])

for tweet in tweets:
    hash_value = binascii.crc32(tweet.encode('utf-8')) & 0xffffffff
    data.append(hash_value)

stream = Counter(data)

# Calculate the second moment for a particular data stream using Σi^2
second_moment = sum(value**2 for value in stream.values())
print('Truth second moment for',len(data),'data stream: ',second_moment)


def AMSEstimate(num_samples, reserv):
    # Add k distinct items to our dictionary 
    dict = {}
    while len(dict) < num_samples:
        r = random.randrange (0,len(reserv),1)
        if reserv[r] not in dict.keys():
            dict[reserv[r]] = r
    k_sum = 0
    for k, v in dict.items():
        #print('k,v',k,v)
        item_counter = reserv[v:].count(reserv[dict[k]])
        item_estimation = len(reserv) * (2 * item_counter - 1)
        k_sum += item_estimation
    return (k_sum / num_samples)

result = []
sample_counter = 0
for item in data:
    N += 1
    if len(result) < K:
        result.append(item)
        sample_counter += 1
    else:
        s = int(random.random() * N)
        if s < K:
            result[s] = item
    #print('len result',len(result))        
    if sample_counter > 10:
        print('2-nd moment estimation using the AMS algorithm: ',AMSEstimate(10,result))

Truth second moment for 5374 data stream:  5716
2-nd moment estimation using the AMS algorithm:  11.0
2-nd moment estimation using the AMS algorithm:  12.0
2-nd moment estimation using the AMS algorithm:  13.0
2-nd moment estimation using the AMS algorithm:  14.0
2-nd moment estimation using the AMS algorithm:  15.0
2-nd moment estimation using the AMS algorithm:  16.0
2-nd moment estimation using the AMS algorithm:  17.0
2-nd moment estimation using the AMS algorithm:  18.0
2-nd moment estimation using the AMS algorithm:  19.0
2-nd moment estimation using the AMS algorithm:  20.0
2-nd moment estimation using the AMS algorithm:  21.0
2-nd moment estimation using the AMS algorithm:  22.0
2-nd moment estimation using the AMS algorithm:  23.0
2-nd moment estimation using the AMS algorithm:  24.0
2-nd moment estimation using the AMS algorithm:  25.0
2-nd moment estimation using the AMS algorithm:  26.0
2-nd moment estimation using the AMS algorithm:  27.0
2-nd moment estimation using the A

In [71]:
#AMS Algorithm k-moment for k = 2 without Reservoir Sampling
import random
import string
import binascii
import operator
from collections import Counter


tweets = []
data = []
#Read tweet id's from csv file
with open('output_tweets.csv','r') as f:
    contents = csv.reader(f)
    for row in contents:
        tweets.append(row[0])

for tweet in tweets:
    hash_value = binascii.crc32(tweet.encode('utf-8')) & 0xffffffff
    data.append(hash_value)

stream = Counter(data)

# Calculate the second moment for a particular data stream using Σi^2
second_moment = sum(value**2 for value in stream.values())
print('Truth second moment for the current stream: ',second_moment)


def AMSEstimate(num_samples):
    # Add k distinct items to our dictionary 
    dict = {}
    while len(dict) < num_samples:
        r = random.randrange (0,len(data),1)
        if data[r] not in dict.keys():
            dict[data[r]] = r
    k_sum = 0
    for k, v in dict.items():
        #print('k,v',k,v)
        item_counter = data[v:].count(data[dict[k]])
        item_estimation = len(data) * (2 * item_counter - 1)
        k_sum += item_estimation
    return (k_sum / num_samples)

print('2-nd moment estimation using the AMS algorithm: ',AMSEstimate(100))

Truth second moment for the current stream:  24052
2-nd moment estimation using the AMS algorithm:  19955.52


In [39]:
#Find text contents of most frequent tweet
freq_dict = Counter(tweets)
most_freq_tweet_id = max(freq_dict, key=freq_dict.get)
print('Most frequent tweet id: ',most_freq_tweet_id)

filePath = 'C:/Users/iTTaste/tweet.json'

fo = open(filePath, "r")
lines = fo.readlines()

#print('Total tweets: ',len(lines))
for line in lines:
    try:
        tweet = json.loads(line)
        if 'retweeted_status' in tweet:
            if str(most_freq_tweet_id) == str(tweet['retweeted_status']['id']):
                print('Found',tweet['text'])
                break
            #print(tweet['retweeted_status']['id'])
    except ValueError:
        pass

fo.close()

Most frequent tweet id:  1213824664734031877
Found RT @BTS_twt: 아미!! 오늘 또 대상 받았어요
새해부터 아미랑 함께 좋은 상 받아서 너무 행복행복해요 고마워요 아미 하트 https://t.co/BRBVslOmxI
